In [ ]:
!nvidia-smi

# EYE GAZE SHARED TASK

In [ ]:
import numpy as np
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import r2_score

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

### DOWNLOAD DATA

In [ ]:
train_data_file_path = "/content/drive/My Drive/CMCL Shared Task/preprocessed_dataset.csv"
test_data_file_path = "/content/drive/My Drive/CMCL Shared Task/preprocessed_test_dataset.csv"

#file_path = "/content/drive/MyDrive/datasets/EyeGazeSharedTask/trial_data.csv"

train_data = pd.read_csv(train_data_file_path)
test_data = pd.read_csv(test_data_file_path)
df = train_data.copy()
test_df = test_data.copy()

In [ ]:
print("Train Data Shape = ", train_data.shape)
print("Test Data Shape = ", test_data.shape)
train_data

### BASIC MODEL PLAN

In [ ]:
#@title
import zipfile
larger_file_path = "/content/drive/My Drive/CMCL Shared Task/glove.840B.300d.zip"
smaller_file_path = "/content/drive/My Drive/CMCL Shared Task/glove.6B.200d.txt"

def load300GloveModel(File):
    file_path = "glove.840B.300d.txt"

    print("Unzipping File...")
    with zipfile.ZipFile(File, "r") as zip:
      zip.extractall()
    print("Finished unzipping File")
    
    print("Loading Glove Model")
    f = open(file_path,'r')
    gloveModel = {}
    for line in f:
        splitLines = line.split(' ')
        try:
          word = splitLines[0]
          wordEmbedding = np.array([float(value) for value in splitLines[1:]])
          gloveModel[word] = wordEmbedding
        except:
          print("Error encountered. Skipping word ", word)
    print(len(gloveModel)," words loaded!")
    return gloveModel

def load200GloveModel(File):
  f = open(File,'r')
  gloveModel = {}
  for line in f:
      splitLines = line.split(' ')
      try:
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word] = wordEmbedding
      except:
        print("Error encountered. Skipping word ", word)
  print(len(gloveModel)," words loaded!")
  return gloveModel

#glove_embedding_dict = load300GloveModel(larger_file_path)
glove_embedding_dict = load200GloveModel(smaller_file_path)

In [ ]:
df_data = pd.read_csv("/content/drive/My Drive/CMCL Shared Task/Sentences_maxlen.csv")
sentences = list(df_data["sentences"])
MAX_LEN = df_data["max_len"][0]

test_df_data = pd.read_csv("/content/drive/My Drive/CMCL Shared Task/Test_Sentences_maxlen.csv")
test_sentences = list(test_df_data["sentences"])
test_MAX_LEN = test_df_data["max_len"][0]

In [ ]:
VOCAB_SIZE = 400000
BATCH_SIZE = 32
N_FEATURES = 13
N_TARGETS = 5

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(sentences)

In [ ]:
word_index = tokenizer.word_index
print(len(word_index))
embedding_matrix = np.zeros((len(word_index) + 1, 200)) # words not found in embedding index will be all-zeros.
for word, i in word_index.items():
  embedding_vector = glove_embedding_dict.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector


In [ ]:
tokens = tokenizer.texts_to_sequences(sentences)
test_tokens = tokenizer.texts_to_sequences(test_sentences)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

pad_sent = pad_sequences(tokens, maxlen=MAX_LEN)
test_pad_sent = pad_sequences(test_tokens, maxlen=MAX_LEN)


























































### PREPROCESS DATA

#### REMOVE EOS TOKEN

In [ ]:
#@title
# FUNCTION TO REMOVE THE <EOS> TOKEN IN THE DATASET

def remove_eos(df):
  cnt = 1
  endword = []
  for i in range(df.shape[0]-1):
    if (df.loc[i+1, "sentence_id"] == cnt):
      df.loc[i, "word"] = df.loc[i, "word"][:-5]   # Remove <EOS> for the last word of each sentence.
      cnt += 1 
      endword.append(1)
    else:
      endword.append(-1)

  s = df.loc[df.shape[0] - 1, "word"]              # Remove <EOS> for last element separately
  df.loc[df.shape[0] - 1, "word"] = s[:-6] 
  endword.append(1)
  df["endword"] = endword
  return df

### NO OF CHARS FOR EACH WORD

In [ ]:
#@title
# FUNCTION TO CALCULATE THE NUMBER OF CHARACTERS PER WORD
# ADDS THE DATA IN A NEW COLUMN

def char_per_word(df):
  n_chars = []
  for word in df.word:
    n_chars.append(len(str(word)))
  df["n_chars"] = n_chars
  return df

### NO OF CHARS OF WORD - LEMMATIZED WORD

In [ ]:
#@title
# FUNCTION TO CALCULATE THE DIFFERENCE BETWEEN NUMBER OF CHARACTERS IN WORD AND LEMMATIZED WORD
# ADDS AS A NEW COLUMN

from nltk import WordNetLemmatizer

Lemmatizer = WordNetLemmatizer()

def char_per_lemmatized_word(df):
  n_chars = []
  for word in df.word:
    n_chars.append(len(str(word)) - len(Lemmatizer.lemmatize(word)))
  df["n_char_lemmatized"] = n_chars
  return df

### STOP WORD OR NOT

In [ ]:
#@title
import nltk
nltk.download("stopwords")
nltk.download("wordnet")

stopwords = nltk.corpus.stopwords
stop_words = stopwords.words("english")

In [ ]:
#@title
# FUNCTION TO ASSERT WHETHER A WORD IS STOPWORD  OR NOT
# ADDS THE DATA IN A NEW COLUMN

def add_stopword_check(df):
  if_stopword = []
  for word in df.word:
    if word in stop_words:
      if_stopword.append(1)
    else:
      if_stopword.append(-1)

  df["stopword"] = if_stopword
  return df

### NUMBER OR NOT

In [ ]:
#@title
# FUNCTION TO DEFINE WHETHER IT IS A NUMBER OR NOT
# ADDS THE DATA AS A NEW COLUMN

def add_number_check(df):
  if_number = []
  for word in df.word:
    if word.isdigit():
      if_number.append(1)
    else:
      if_number.append(-1)

  df["number"] = if_number
  return df

### TF IDF CALCULATION

In [ ]:
#@title
# FUNCTION TO CALCULATE THE TFIDF OF THE TRAINING DATASET
# ADDS THE DATA IN A NEW COLUMN
# ALSO RETURNS A LIST OF THE SENTENCES

import re
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words=None)
bad_words = []
punc = string.punctuation

def remove_punc(word):
  table = str.maketrans('', '', punc)
  return word.translate(table)

def calc_tfidf(df):
  n = np.array(df["sentence_id"])[-1]
  sentences = []
  sentence_tokens = []
  tf_idfs = []
  MAX_LEN = 0
  for i in range(n+1):   
    temp_df = df[df.sentence_id == i]
    sentence = (' ').join(temp_df.word)
    MAX_LEN = max(MAX_LEN, len(sentence))
    sentences.append(sentence)
    sentence_tokens.append([np.array(temp_df.word)])
  tf_idf = vectorizer.fit_transform(sentences)
  for i, word in enumerate(df.word):
    try:
      tf_idfs.append(tf_idf.toarray()[df["sentence_id"][i]][vectorizer.get_feature_names().index(remove_punc(word.lower()))])
    except:
      bad_words.append(word)
      if word in ["a", "A"]:
        tf_idfs.append(0.8)
      else:
        tf_idfs.append(0.01)
  df["tf_idf"] = tf_idfs


  return sentence_tokens, df, bad_words, MAX_LEN

In [ ]:
#@title
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize 
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse = False)

def pos_tag_func(df):
  tags = []
  for word in df.word:
    if word not in string.punctuation:
      tag = nltk.pos_tag(word)[0][1]
    else:
      tag = "PUNC"
    tags.append(tag)
  df["tags"] = tags
  tag_transform = pd.DataFrame(enc.fit_transform(np.array(df.tags).reshape(-1, 1)))
  df = pd.concat((df, tag_transform), axis = 1)
  return df

In [ ]:
#@title
def use_transformed_GPT(df):
  df["GPT"] = df["TRT"] - df["GPT"]
  return df

In [ ]:
# FUNCTION TO PERFORM ALL PREPROCESSING STEPS

def preprocess(df):
  return calc_tfidf(use_transformed_GPT(pos_tag_func(add_number_check(add_stopword_check(char_per_lemmatized_word(char_per_word(remove_eos(df))))))))

sentence_tokens, df, bad_words, MAX_LEN = preprocess(train_data)

df1 = df.copy()
df1

In [ ]:
print("No of words unaccounted for = ", len(bad_words))

In [ ]:
# FUNCTION TO NORMALISE ALL THE TARGET VALUES

from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()

def standardize_target(df):
  keys = df.keys()[3:8]
  new_df = pd.DataFrame(std_scaler.fit_transform(df.iloc[:,3:8]), columns = keys)
  df.update(new_df)
  return 
standardize_target(df)
df

### FORM TARGETS

In [ ]:
# FUNCTION TO FORM TARGETS AND OTHER FEATURES IN SEPARATE DATA STRUCTURES

def create_glove_embedding(sentence, embedding_dict):
  tokens = []
  seq_len = len(sentence)
  pre_padding = [list(embedding_dict["pad"]) for i in range(max(0, MAX_LEN - seq_len))]
  att_mask = [[0, 0, 0, 0, 0] for i in range(MAX_LEN - seq_len)] + [[1, 1, 1, 1, 1] for j in range(seq_len)]
  tokens += pre_padding
  for word in sentence:
    try:
      tokens.append(list(embedding_dict[word]))
    except:
      tokens.append(list(embedding_dict["unk"]))
  return tokens, att_mask

def form_targets(sentence_tokens, df, MAX_LEN, train = True):
  start_pos = 800
  if train:
    start_pos = 0
  n = np.array(df["sentence_id"])[-1]
  targets = []
  tags = []
  embeddings = []
  attention_masks = []
  features = {"n_chars" : [],
              "stopword" : [],
              "number" : [],
              "endword":[],
              "n_char_lemmatized" : [],
              "tf_idf" : [],
              }

  for i in range(start_pos, n+1):
    feature = {}
    actual_features = {}
    temp_df = df[df.sentence_id == i]
    sentence_tokens = [w for w in temp_df.word]
    
    #attention_mask = [0 for j in range(MAX_LEN - len(sentence_tokens))] + [1 for j in range(len(sentence_tokens))]
    
    if train:
      target = [[0, 0, 0, 0, 0] for j in range(MAX_LEN - len(sentence_tokens))]
      actual_targets = [list(x) for x in np.array(temp_df.iloc[:, 3:8])]
      target += actual_targets
    
    embedding, att_mask = create_glove_embedding(sentence_tokens, glove_embedding_dict)
    attention_masks.append(att_mask)

    for key in features.keys():
      feature[key] = [0 for j in range(MAX_LEN - len(sentence_tokens))]
      actual_features[key] = list(np.array(temp_df.loc[:, key]))
      feature[key] += actual_features[key]

    tag = [[0 for j in range(7)] for k in range(MAX_LEN - len(sentence_tokens))]
    actual_tag = [list(x) for x in np.array(temp_df.iloc[:, -8:-1])]
    tag += actual_tag

  

    for key in features.keys():
      features[key].append(feature[key]) 
    if train:
      targets.append(target) 
    tags.append(tag)
    embeddings.append(embedding)

  return targets, features, tags, attention_masks, embeddings

targets, features, tags, attention_masks, embeddings = form_targets(sentences, df, MAX_LEN)
test_targets, test_features, test_tags, test_attention_masks, test_embeddings = form_targets(test_sentences, test_df, MAX_LEN, False)

print("Target Shape = ", np.array(targets).shape)

In [ ]:
print(np.array(features["n_chars"]).shape)

In [ ]:
np.array(attention_masks).shape

In [ ]:
np.array(tags).shape

In [ ]:
def create_model_inputs(features, tags):
  extra_features = tf.zeros(shape = (np.array(tags).shape[0], 65, 0), dtype = tf.float32)
  for key in features.keys():
    t = tf.convert_to_tensor(features[key],dtype = tf.float32)
    t = tf.expand_dims(t, axis = 2)
    extra_features = tf.concat((extra_features, t), axis = 2)
  extra_features = tf.concat((extra_features, tags), axis = 2)
  return extra_features

In [ ]:
extra_features = create_model_inputs(features, tags)
test_extra_features = create_model_inputs(test_features, test_tags)
extra_features.shape

In [ ]:
test_extra_features.shape

In [ ]:
embeddings = tf.convert_to_tensor(embeddings, dtype = tf.float32)
test_embeddings = tf.convert_to_tensor(test_embeddings, dtype = tf.float32)
embeddings.shape

In [ ]:
test_embeddings.shape

In [ ]:
targets = tf.convert_to_tensor(targets, dtype = tf.float32)
targets.shape

In [ ]:
np.array(attention_masks).shape

In [ ]:
np.array(test_attention_masks).shape

In [ ]:
targets.numpy()

### IMPORT TOKENIZERS AND MODELS

In [ ]:
from sklearn.model_selection import train_test_split

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

### EARLY STOPPING CLASS

### FORMING CLASS OF DATA

In [ ]:
device = "cpu"
with tf.device(device):
  # INPUT LAYER

  embedding_input = tf.keras.Input(shape = (MAX_LEN, ), name = "Embedding_Input")
  features_input = tf.keras.Input(shape = (MAX_LEN, N_FEATURES), name = "Features_Input")
  attention_mask_input = tf.keras.Input(shape = (MAX_LEN, N_TARGETS), name = "Attention_mask_Input")

  #PASS THROUGH EMBEDDING
  GloVe = tf.keras.layers.Embedding(len(word_index) + 1, 200, weights=[embedding_matrix], input_length=MAX_LEN, trainable=True)
  embedding_output = GloVe(embedding_input)

  # PASS THROUGH LANGUAGE LSTMS

  lang_input_layer = tf.keras.layers.Dense(256, activation = "relu", name = "Lang_Input_layer")
  lang_forward_layer = tf.keras.layers.LSTM(128, return_sequences = True, name = "GRU_Layer")
  lang_bilayer = tf.keras.layers.Bidirectional(lang_forward_layer, merge_mode = "concat", name = "BiGRU_Layer")
  lang_out_layer = tf.keras.layers.Dense(256, activation = "relu", name = "Lang_Output_Layer")


  lang_input = lang_input_layer(embedding_output)
  lang_hidden_output = lang_bilayer(lang_input)
  lang_hidden_output = lang_bilayer(lang_hidden_output)
  lang_hidden_output = lang_bilayer(lang_hidden_output)
  lang_output = lang_out_layer(lang_hidden_output)

  # PASS THROUGH FEATURE LSTMS

  feature_forward_layer = tf.keras.layers.LSTM(128, return_sequences = True, name = "Feature_GRU_Layer")
  feature_bilayer = tf.keras.layers.Bidirectional(feature_forward_layer, merge_mode = "concat", name = "Feature_BiGRU_Layer")
  feature_dense_layer = tf.keras.layers.Dense(256, activation = "relu", name = "Feature_Dense_Layer")
  feature_out_layer = tf.keras.layers.Dense(256, activation = "relu", name = "Feature_Output_Layer")

  feature_input = feature_dense_layer(features_input)    
  feature_hidden_output = feature_bilayer(feature_input)
  feature_hidden_output = feature_bilayer(feature_hidden_output)
  feature_output = feature_out_layer(feature_hidden_output)

  # TAKE A MEAN
  output = (lang_output+feature_output)/2

  # PASS THROUGH HEAD LAYER

  head_forward_layer = tf.keras.layers.LSTM(128, return_sequences = True, name = "Head_GRU_Layer")
  head_bilayer = tf.keras.layers.Bidirectional(head_forward_layer, merge_mode = "concat", name = "Head_BiGRU_Layer")
  head_dense = tf.keras.layers.Dense(128, activation = "relu", name = "Head_Dense_Layer")
  head_drop = tf.keras.layers.Dropout(rate = 0.1)
  head_out = tf.keras.layers.Dense(5, activation = "relu", name = "Head_FinalOutput_Layer")

  head_hidden_output = head_bilayer(output)
  head_hidden_output= head_bilayer(head_hidden_output)
  head_output = head_drop(head_dense(head_hidden_output))
  output = head_out(head_output)

  output = tf.math.multiply(output, attention_mask_input)

  model = tf.keras.Model(inputs = [embedding_input, features_input, attention_mask_input], outputs = output, name = "EyeGazeModel")

In [ ]:
def custom_r2(pred, true):
  ssr = tf.math.reduce_sum(tf.math.square(true - pred))
  sst = tf.math.reduce_sum(tf.math.square(true - tf.math.reduce_mean(true)))
  a = 1-ssr/sst
  return a

In [ ]:
from sklearn.metrics import r2_score

loss_fn = tf.keras.losses.MAE
adam = tf.keras.optimizers.Adam(learning_rate = 1e-3, beta_1 = 0.902, beta_2 = 0.999)

model.compile(optimizer = adam, loss = loss_fn, metrics = custom_r2)
model.summary()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", min_delta = 0.0001, patience = 8, verbose = 2, restore_best_weights = True)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='tflstm_best_model.hdf5', monitor='val_loss', save_best_only=True, save_weights_only=True)

history = model.fit(x = [pad_sent, extra_features.numpy(), np.array(attention_masks)], y = targets.numpy(), batch_size = 32, shuffle = True, epochs = 100, verbose = 1, validation_split = 0.25, callbacks=[callback, model_checkpoint])

In [ ]:
train_preds = model.predict(x = [pad_sent, extra_features.numpy(), np.array(attention_masks)], batch_size = 1, verbose = 1)

In [ ]:
print(r2_score(train_preds[:,:,0], targets[:,:,0]))
print(r2_score(train_preds[:,:,1], targets[:,:,1]))
print(r2_score(train_preds[:,:,2], targets[:,:,2]))
print(r2_score(train_preds[:,:,3], targets[:,:,3]))
print(r2_score(train_preds[:,:,4], targets[:,:,4]))

In [ ]:
preds = model.predict(x = [test_pad_sent, test_extra_features.numpy(), np.array(test_attention_masks)], batch_size = 1, verbose = 1)

In [ ]:
plt.plot(history.history["loss"], c = "g", label = "Train Loss")
plt.plot(history.history["val_loss"], c = "r", label = "Val Loss")
plt.legend()
plt.show()

In [ ]:
model.save(filepath = "/content/drive/My Drive/CMCL Shared Task/TFBiLSTM_DE_Features.h5")

### TRAINING

In [ ]:
from collections import defaultdict

history = defaultdict(list)
tolerance = 0
best = {}
best = {"val_loss" : 10000}
early_stopping = EarlyStopping(patience = 20, verbose = True)

for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 120)

  train_loss, train_r2 = train_epoch(model,
    train_data_loader,    
    loss_fn, 
    optimizer, 
    device)
  

  print(f'Train loss {train_loss} and Train R2 {train_r2}')

  val_loss, val_r2 = eval_model(
    model,
    val_data_loader,
    loss_fn, 
    device
  )

  print(f'Val loss {val_loss} and Val R2 {val_r2}')
  print()

  history = update(history, train_loss, val_loss, train_r2, val_r2)
  
  if val_loss < best["val_loss"]:
    best = save(history, best)
  
  early_stopping(val_loss, model)
  if early_stopping.early_stop:
    print("Stopped Early at at Epoch ", epoch+1)
    break
  model.load_state_dict(torch.load('checkpoint.pt'))


In [ ]:
plt.plot(history["train_loss"], c = "r", label = "Train Loss")
plt.plot(history["val_loss"], c = "g", label = "Validation Loss")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("L1 Loss")
plt.show()

In [ ]:
plt.plot(history["train_nFix"], c = "r", label = "Train Loss")
plt.plot(history["val_nFix"], c = "g", label = "Validation Loss")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("nFix R2")
plt.show()

In [ ]:
plt.plot(history["train_FFD"], c = "r", label = "Train Loss")
plt.plot(history["val_FFD"], c = "g", label = "Validation Loss")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("FFD R2")
plt.show()

In [ ]:
plt.plot(history["train_GPT"], c = "r", label = "Train Loss")
plt.plot(history["val_GPT"], c = "g", label = "Validation Loss")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("GPT R2")
plt.show()

In [ ]:
plt.plot(history["train_TRT"], c = "r", label = "Train Loss")
plt.plot(history["val_TRT"], c = "g", label = "Validation Loss")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("TRT R2")
plt.show()

In [ ]:
plt.plot(history["train_fixProp"], c = "r", label = "Train Loss")
plt.plot(history["val_fixProp"], c = "g", label = "Validation Loss")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("fixProp R2")
plt.show()

In [ ]:
train_arr = np.array([best["val_nFix"], best["val_FFD"], best["val_GPT"], best["val_TRT"], best["val_fixProp"]])
val_arr = np.array([best["val_nFix"], best["val_FFD"], best["val_GPT"], best["val_TRT"], best["val_fixProp"]])

train_mean = np.mean(train_arr)
train_std = np.std(train_arr)
train_M = np.max(train_arr)
train_m = np.min(train_arr)
val_mean = np.mean(val_arr)
val_std = np.std(val_arr)
val_M = np.max(val_arr)
val_m = np.min(val_arr)

display_data = [[best["train_loss"], best["train_nFix"], best["train_FFD"], best["train_GPT"], best["train_TRT"], best["train_fixProp"], train_mean, train_std, train_M, train_m],
 [best["val_loss"], best["val_nFix"], best["val_FFD"], best["val_GPT"], best["val_TRT"], best["val_fixProp"], val_mean, val_std, val_M, val_m]]

display_df = pd.DataFrame(display_data, columns = ["L1Loss", "nFix", "FFD", "GPT", "TRT", "fixProp", "Mean", "Std Deviation", "Max", "Min"], index = ["Train", "Val"]) 

In [ ]:
display_df

In [ ]:
display_data2 = [[history["train_loss"][-1], history["train_nFix"][-1], history["train_FFD"][-1], history["train_GPT"][-1], history["train_TRT"][-1], history["train_fixProp"][-1]], 
                 [history["val_loss"][-1], history["val_nFix"][-1], history["val_FFD"][-1], history["val_GPT"][-1], history["val_TRT"][-1], history["val_fixProp"][-1]]]
display_df2 = pd.DataFrame(display_data2, columns = ["L1Loss", "nFix", "FFD", "GPT", "TRT", "fixProp"], index = ["Train ", "Val "]) 

In [ ]:
display_df2

In [ ]:
history["train_loss"][0], history["val_loss"][0]

In [ ]:
save_file_path = "/content/drive/My Drive/CMCL Shared Task/LSTM_Features_20.pth"
torch.save(model.state_dict(), save_file_path)

In [ ]:
del model

In [ ]:
'''
IDEAS:
Training :-
1. Sentence Formation.
2. BERT tokenize.
3. Base BERT Model -> if encoded value == 0 -> 5 output Dense Layer.
4. MAE metric for loss calculation.

Test :-
1. Sentence Formation.
2. BERT Tokenize.
3. Base BERT Model  -> if encoded value == 0 -> 5 output Dense Layer.
4. Order is maintained and predictions are pasted on the csv file.
'''

In [ ]:
'''
trainer = Engine(train_epoch)
train_evaluator = Engine(train_epoch)
validation_evaluator = Engine(val_epoch)

Loss(loss_fn).attach(train_evaluator, "l1")
Loss(loss_fn).attach(validation_evaluator, "l1")

def score_function(engine):
    val_loss = engine.state.metrics['nll']
    return val_loss

handler = EarlyStopping(patience = 10, score_function=score_function, trainer = trainer)
validation_evaluator.add_event_handler(Events.COMPLETED, handler)

def log_training_results(engine):
    train_evaluator.run(train_data_loader)
    metrics = train_evaluator.state.metrics
    pbar.log_message(
        "Training Results - Epoch: {} \nMetrics\n{}"
        .format(engine.state.epoch, pprint.pformat(metrics)))
    
def log_validation_results(engine):
    validation_evaluator.run(val_data_loader)
    metrics = validation_evaluator.state.metrics
    metrics = validation_evaluator.state.metrics
    pbar.log_message(
        "Validation Results - Epoch: {} \nMetrics\n{}"
        .format(engine.state.epoch, pprint.pformat(metrics)))
    pbar.n = pbar.last_print_n = 0

trainer.add_event_handler(Events.EPOCH_COMPLETED, log_validation_results)

checkpointer = ModelCheckpoint('checkpoint', 'textcnn', save_interval=1, n_saved=2, create_dir=True, save_as_state_dict=True)

best_model_save = ModelCheckpoint(
    'best_model', 'textcnn', n_saved=1,
    create_dir=True, save_as_state_dict=True,
    score_function=score_function)
trainer.add_event_handler(Events.EPOCH_COMPLETED, checkpointer, {'textcnn': model})
validation_evaluator.add_event_handler(Events.EPOCH_COMPLETED, best_model_save, {'textcnn': model})


trainer.run(train_data_loader, max_epochs=120)
'''